In [45]:
from bs4 import BeautifulSoup
import urllib.request, urllib.error, urllib.parse
import pandas as pd
from selenium import webdriver

In [46]:
cnn_url = 'https://edition.cnn.com'
world_url = 'https://edition.cnn.com/world'
politics_url = 'https://edition.cnn.com/politics'
health_url = 'https://edition.cnn.com/health'
entertainment_url = 'https://edition.cnn.com/entertainment'

In [47]:
def getParseUrl(link):
    driver = getDriver()
    driver.get(link)
    webContent = driver.page_source
    
    data = BeautifulSoup(webContent, 'html.parser')
    driver.close()
    return data

In [62]:
def getParseUrlStatic(link):
    response = urllib.request.urlopen(link)
    webContent = response.read()
    
    data = BeautifulSoup(webContent, 'html.parser')
    return data

In [63]:
def getNewsLinks(data):
    headlines = data.find_all(class_ = "cd__headline")
    links = []
    for headline in headlines:
        link = headline.a["href"]
        if link[:5] != 'https':
            link = cnn_url + link
        links.append(link)
    return links

In [64]:
def getDriver():
    driver = webdriver.Chrome(executable_path=r'C:\Users\DELL\Downloads\chromedriver_win32\chromedriver')
    return driver

In [77]:
def getStringFromTags(tag):
    data = tag.contents
    string = ""
    for content in data:
        try:
            string += content
        except:
            string += getStringFromTags(content)
    return string

In [69]:
def getNewsFromLink(link, genre):
    data = getParseUrlStatic(link)
    news = {}
    news['heading'] = data.h1.contents[0]
    news['content'] = ""
    divs = data.find_all(True, {'class': ["zn-body__paragraph", "Paragraph__component BasicArticle__paragraph BasicArticle__pad", "Paragraph__component"]})
    for div in divs:
        news['content'] += getStringFromTags(div)
    news['link'] = link
    news['genre'] = genre
    return news

In [70]:
def getDataFromUrl(url, genre):
    data = getParseUrl(url)
    links = getNewsLinks(data)
    news = [getNewsFromLink(link, genre) for link in links if link[:23] == 'https://edition.cnn.com']
    return news

In [71]:
news = []
for genre in ['world', 'politics', 'entertainment']:
    url = cnn_url + '/' + genre
    print(genre)
    cur_news = getDataFromUrl(url, genre)
    for new in cur_news:
        news.append(new)

world
politics
entertainment


In [72]:
len(news)

168

In [75]:
pd.DataFrame(news)

,heading,content,link,genre
0,New Lebanon PM seeks 'quick fixes' to help his...,(CNN)New Prime Minister Najib Mikati urged Le...,https://edition.cnn.com/2021/09/17/middleeast/...,world
1,What are nuclear-powered submarines and how do...,Hong Kong (CNN)With Australia making a deal w...,https://edition.cnn.com/2021/09/17/australia/n...,world
2,Canada's cliffhanger election is heating up. S...,"Carp, Ontario (CNN)The sky was blue, the famil...",https://edition.cnn.com/2021/09/17/americas/ca...,world
3,Video of child taken to quarantine alone gains...,,https://edition.cnn.com/videos/world/2021/09/1...,world
4,How Europe's hospitals are faring in the face ...,London (CNN)Much of Europe has opened up to in...,https://edition.cnn.com/2021/09/18/europe/euro...,world
...,...,...,...,...
163,'Impeachment: American Crime Story' dissects t...,"(CNN)The ""American Crime Story"" banner launch...",https://edition.cnn.com/2021/08/31/entertainme...,entertainment
164,See what's streaming in September,A look at some of the series and films streami...,https://edition.cnn.com/2021/08/31/entertainme...,entertainment
165,'Bob Ross' paints a complex portrait of the ar...,(CNN)Add the late PBS painter Bob Ross to the...,https://edition.cnn.com/2021/08/25/entertainme...,entertainment
166,When some of your favorite shows are returning...,"(CNN)The fall TV season is almost here, so wh...",https://edition.cnn.com/2021/09/01/entertainme...,entertainment
